In [5]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import storage

from datetime import datetime
import matplotlib.pyplot as plt
import pickle


from IPython.display import display, HTML
import os


from ficc.data.process_data import process_data
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS
from ficc.utils.gcp_storage_functions import upload_data, download_data
from ficc.utils.nelson_siegel_model import yield_curve_level
from ficc.utils.diff_in_days import diff_in_days_two_dates
from ficc.utils.auxiliary_variables import NUM_OF_DAYS_IN_YEAR
from ficc.utils.adding_flags import add_bookkeeping_flag, add_same_day_flag, add_ntbc_precursor_flag, add_replica_flag

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/ml_models/sequence_predictors/ahmad_creds.json"
storage_client = storage.Client()
bq_client = bigquery.Client()

In [7]:
import gcsfs
fs = gcsfs.GCSFileSystem(project='eng-reactor-287421')
with fs.open('ficc_training_data_latest/processed_data_2022-10-21-17:41.pkl') as f:
    data = pd.read_pickle(f)

In [8]:
%%time
data = add_replica_flag(data)

CPU times: user 3h 8min 37s, sys: 1min 15s, total: 3h 9min 52s
Wall time: 3h 11min 52s


In [9]:
%%time
data = add_bookkeeping_flag(data)

CPU times: user 53min 48s, sys: 37 s, total: 54min 25s
Wall time: 55min 8s


In [10]:
data = add_same_day_flag(data)

/opt/conda/lib/python3.7/site-packages/ficc/utils/adding_flags.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[flag_name] = False


In [11]:
add_ntbc_precursor_flag(data)

,MSRB_maturity_date,MSRB_coupon_rate,msrb_cusip,yield_spread,calc_price,price_to_next_call,price_to_par_call,price_to_maturity,calc_date,price_delta,...,ficc_ycl_to_refund,last_trade_date,last_ficc_ycl_to_maturity,last_ficc_ycl_to_next_call,last_ficc_ycl_to_par_call,last_ficc_ycl_to_refund,is_replica,is_bookkeeping,is_same_day,ntbc_precursor
0,2026-06-15,2.000000000,86476PSV1,56.184457,94.191,97.269,97.269,94.191,2026-06-15,0.001,...,NaN,2022-10-14,3.149726,3.125085,3.125085,NaN,False,False,True,False
1,2036-10-01,0E-9,592643BE9,77.396313,52.288,0.000,0.000,0.000,2036-10-01,0.004,...,NaN,2022-10-13,3.934756,NaN,NaN,NaN,False,False,False,False
2,2026-12-01,2.125000000,94244CAG7,97.787294,92.409,97.807,97.807,92.409,2026-12-01,0.002,...,NaN,2022-10-14,3.169599,3.141609,3.141609,NaN,False,False,True,False
3,2038-12-01,4.000000000,91739RDN4,40.526398,94.307,96.289,96.289,94.307,2038-12-01,0.000,...,NaN,2022-10-13,4.094156,3.536483,3.536483,NaN,False,False,False,True
4,2038-12-01,4.000000000,91739RDN4,40.526398,94.307,96.289,96.289,94.307,2038-12-01,0.000,...,NaN,2022-10-13,4.094156,3.536483,3.536483,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296713,2042-12-01,5.000000000,944514NS3,40.140010,101.874,101.874,101.874,148.373,2022-12-01,0.001,...,NaN,2022-03-30,2.855105,1.699600,1.699600,NaN,False,False,False,False
5296714,2026-08-01,4.000000000,254498FH6,6.970756,107.972,0.000,0.000,0.000,2026-08-01,0.001,...,NaN,2022-03-24,1.992905,NaN,NaN,NaN,False,False,False,False
5296715,2026-08-01,4.000000000,254498FH6,32.670756,106.872,0.000,0.000,0.000,2026-08-01,0.001,...,NaN,2022-03-24,1.992905,NaN,NaN,NaN,False,False,False,False
5296716,2026-12-01,5.000000000,13067WCQ3,3.900978,111.822,111.822,111.822,113.179,2026-06-01,0.001,...,NaN,2022-03-15,2.019444,1.982869,1.982869,NaN,False,False,False,False


In [12]:
data.to_pickle('processed_data_2022-10-21-17:41-flags.pkl')